# Dictionary / Map

Während Datensätze durch *Vergleiche* ihrer Schlüssel gesucht werden, geschieht dies beim **Hashing** (to hash = klein zerhacken) durch deren *Berechnung*. Es ist eine Methode zur dynamischen Verwaltung von Daten (value), die über einen Schlüssel (key) angesprochen werden. Ein Dictionary, auch Map genannt, ist ein Abstrakter Datentyp, der in der Informatik sehr wichtig ist und sehr häufig zum Einsatz kommt. Er ist in allen gängigen Programmiersprachen implementiert.

Ein Dictionary besteht aus Schlüssel-Wert-Paaren (key-value pairs). Dabei wird von einem Schlüssel auf einen Wert abgebildet. Es gibt drei wesentliche Operationen, nämlich __insert__, __get__ und __remove__.

__insert(key, value)__:

Die Insert-Operation fügt ein Paar aus einem Schlüssel und einem Wert in die Datenstruktur ein. Sowohl Schlüssel, als auch der Wert, auf den abgebildet werden soll, können von jedem erdenklichen Datentyp sein.

__get(key)__:

Diese Operation gibt den Wert, auf den vom angegebenen Schlüssel abgebildet wird, zurück. Befindet sich kein Eintrag mit diesem Schlüssel im Dictionary, so wird __null__ zurückgegeben.

__remove(key)__:

Diese Operation entfernt einen Eintrag mit dem gegebenen Schlüssel aus dem Dictionary.

Die genannten drei Operationen ließen sich mit balancierten Bäumen, beispielsweise einem AVL-Baum, implementieren. Jedoch liegt die Laufzeit für diese Operationen bei einem balancierten Baum in $\mathcal{O}(\log n)$. Ein Ziel dieses Kapitels ist es, eine Datenstruktur zu implementieren, die dies in konstanter Zeit schafft.

# Direct Access Table

Bei diesem Implementierungsversuch weist man, wie bei einem Array, der Datenstruktur einen festen Bereich im Speicher  zu. Nun kann man direkt über den Index auf jeden Slot zugreifen. Sind die Schlüssel nicht-negative ganze Zahlen, so kann man festlegen, dass der Schlüssel immer genau dem Index des Slots entspricht.

In [1]:
m = 10000


class DirectAccessTable:
    def __init__(self):
        self.table = []
        for i in range(m):
            self.table.append(None)

    def insert(self, key, value):
        self.table[hash(key)] = value

    def get(self, key):
        return self.table[hash(key)]

    def remove(self, key):
        self.table[hash(key)] = None


# data type that only allows non-negative integers not exceeding slots size m
class MyDataType:
    def __init__(self, n):
        if 0 <= n < m:
            self.n = n
        else:
            raise Exception('Invalid value')

    def __hash__(self):
        return self.n


direct_access_table = DirectAccessTable()
a = MyDataType(1)
b = MyDataType(125)
c = MyDataType(7632)

direct_access_table.insert(a, 'Hello')
direct_access_table.insert(b, 'World')
direct_access_table.insert(c, '!')

print(direct_access_table.get(a))
print(direct_access_table.get(b))
print(direct_access_table.get(c))
direct_access_table.remove(b)
print(direct_access_table.get(b))


Hello
World
!
None


Dies wirft zwei Probleme auf:
1. Zum einen möchte man eine Datenstruktur haben, bei welcher die Schlüssel von einem beliebigen Datentyp sind. (Es wurde als Bedingung angenommen, dass es sich um nicht-negative ganze Zahlen handelt). 
2. Zum anderen beträgt der Speicheraufwand $\mathcal{O}(\left| U \right|)$. $U$ ist dabei das **Schlüsseluniversum**, d.h. die Menge aller möglichen Schlüssel. Für jeden (auch unverwendeten) Schlüssel wird Speicher reserviert. Dies wäre absolut unpraktikabel.

Die meist viel kleinere **Schlüsselmenge** $K$ ($K \subset U$) ist die Menge aller tatsächlich verwendeten Schlüssel, d.h. die, die sich zum betrachteten Zeitpunkt im Dictionary befinden.


<img src="http://faculty.ycp.edu/~dbabcock/PastCourses/cs360/lectures/images/lecture11/directaddress.png" width="400">

Die Anzahl $n$ der Elemente, die sich momentan in der Datenstruktur befinden, ergibt sich aus der Kardinalität der Schlüsselmenge $\left| K \right|$ .

# Hash Table

Eine Hash Table bzw. Hash Map ist eine Implementation des ADT Dictionary. Dabei wird auch per direkten Slotzugriff, wie bei der Direct Access Table, vorgegangen. Jedoch gibt es eine feste Anzahl $m$ an Slots, wobei gilt $m \ll \left| U \right|$. Nun wird eine Hashfunktion $h: U \to \{0, 1, \dotsc, m-1\}$ benötigt, die vom Schlüsseluniversum auf einen der Slots $\{0, 1, \dotsc, m-1\}$ abbildet. Aus diesem Größenvergleich folgt, dass eine Hashfunktion im Allgemeinen nicht injektiv ist. Eine gute Hashfunktion ist surjektiv, damit alle Schlüssel vorkommen können. Eine einfache Funktion ist beispielsweise $h: k \mapsto k \bmod m$.

Somit wurde der Speicheraufwand deutlich reduziert.

Um das Problem, dass es sich bei den Schlüsseln um etwas anderes als nicht-negative Integers, z.B. Strings, handeln kann, zu lösen, wird eine sogenannte pre-hash Funktion benötigt. Diese ist eine Funktion, die vom Schlüsseluniversum auf eine nicht-negative ganze Zahl abbildet. Theoretisch betrachtet ist dies immer möglich, da alles, was in einem Computer dargestellt wird, diskret und endlich ist. Schließlich könnte man die Bits, durch welche das entsprechende Datum dargestellt wird, als nicht-negativen Integer auffassen und dies als den pre-hash-Wert nehmen. Dies würde jedoch mitunter zu sehr großen Zahlen führen, weshalb man in der Praxis meist bessere pre-hash-Funktionen verwendet. Hat man beispielsweise einen String mit Zeichen aus <tt>a-z</tt>, so könnte man diese Zeichenkette als Zahl im Stellenwertsystem mit der Basis $26$ auffassen.

__Beispiel__:

$h('adf') = 0 \cdot 26^2 + 3 \cdot 26 + 5 \cdot 1 = 83$

Nun kann man mit dem "geprehashten" Wert, der ein nicht-negativer Integer ist, so umgehen, als wäre dieser der eigentliche Schlüssel. 

In der Programmiersprache Java beispielsweise findet dieses Pre-Hashing durch die `hashCode()`-Methode, die jede Klasse implementiert, statt.

## Kollisionen

Durch die Tatsache, dass $h$ wegen $m < \left| U \right|$ im Allgemeinen nicht injektiv ist, entsteht ein neues Problem, nämlich Kollisionen.

Eine **Kollision** unter der Hashfunktion $h$ tritt auf, wenn für $k_i, k_j \in K$ mit $i \neq j$ gilt $h(k_i) = h(k_j)$.

<img src="http://www.cs.fsu.edu/~burmeste/slideshow/images_content/figure12_2.gif" width="450">

Eine Kollision ergibt sich, wenn zwei oder mehrere unterschiedliche Schlüssel den gleichen Hash-Wert haben. Dies hat zur Folge, dass sie sich den gleichen Slot in der Hash Map teilen müssten. Um dieses Problem zu behandeln, gibt es mehrere Verfahren. Im Folgenden werden **Hashing with Chaining** und **Open Addressing** vorgestellt.

## Hashing with Chaining

Bei __Hashing with Chaining__ wird in einem Slot der Hash Table anstatt eines Wertes eine Linked List von Schlüssel-Wert-Paaren gespeichert. Kommt es beim Einfügen zu einer Kollision, so fügt man das Schlüssel-Wert-Paar ans Ende dieser Liste ein. Möchte man nach einem Schlüssel suchen, so muss man durch die Liste an dem entsprechenden Slot iterieren. 

<img src="http://www.cs.fsu.edu/~burmeste/slideshow/images_content/figure12_3.gif" width="450">

Für die Implementation nutzen wir die SinglyLinkedList, die bereits in Kapitel 3 implementiert wurde. Der Einfachkeit halber, wird auf die Implementation der __remove__-Methode verzichtet.

In [2]:
class Node:
    def __init__(self):
        self.value = None
        self.next = None

    def add(self, value, index):
        if index > 1 and self.next is not None:
            return self.next.add(value, index - 1)
        elif index == 1:
            new_node = Node()
            new_node.value = value
            new_node.next = self.next
            self.next = new_node
            return True
        return False


class SinglyLinkedList:
    def __init__(self):
        self.head = Node()
        self.length = 0

    def add(self, value, index=None):
        if index is None:
            index = self.length
        if index == 0:
            new_node = Node()
            new_node.value = value
            new_node.next = self.head
            self.head = new_node
            self.length += 1
        elif 0 < index <= self.length:
            if self.head.add(value, index):
                self.length += 1


class HashTable:
    def __init__(self):
        self.table = []
        for i in range(m):
            self.table.append(SinglyLinkedList())

    # hash function of the hash table, using a really simple one here
    @staticmethod
    def __hash(key):
        return hash(key) % m

    def insert(self, key, value):
        self.table[HashTable.__hash(key)].add((key, value))
        
    def get(self, key):
        node = self.table[HashTable.__hash(key)].head
        while node is not None:
            if node.value is not None and node.value[0] == key:
                return node.value[1]
            node = node.next
        return None
    
    
hash_table = HashTable()

# now our hash table allows us to use any data type as key, in this case a string
hash_table.insert('a', 'Hashing')
hash_table.insert('b', 'Chaining')
hash_table.insert('c', 'with')

print(hash_table.get('a'))
print(hash_table.get('c'))
print(hash_table.get('b'))

Hashing
with
Chaining


Handelt es sich um eine sehr schlechte Hashfunktion, bei der es viele Kollisionen gibt, so kann es passieren, dass (fast) alle Schlüssel den gleichen Hash-Wert haben und somit eine Linked List mit der Länge $\mathcal{O}(n)$ bilden. $n$ ist dabei die Anzahl der Elemente in der Datenstruktur, $n = \left| K \right|$. Die worst-case Komplexität liegt somit für alle Operationen in $\mathcal{O}(n)$. Für gute Hashfunktionen tritt dieser worst-case jedoch mit einer sehr geringen Wahrscheinlichkeit ein. Man kann $\mathcal{O}(\log n)$ im worst-case erreichen, indem man statt Linked Lists ballancierte Bäume verwendet.

## Simple Uniform Hashing

Man spricht von Simple Uniform Hashing, wenn die Hashfunktion $h$ die Schlüsselmenge $K$ uniform zufällig auf die Werte $\{0, 1, \dotsc, m-1\}$ verteilt. *Uniform zufällig* bedeutet, dass jeder Hash-Wert gleich wahrscheinlich auftreten kann, so als ob man den Hash-Wert in einem Zufallsexperiment bestimmt. 

Die Annahme, dass die Verteilung der Hash-Werte uniform ist, tritt in der Praxis nicht ein, jedoch hilft diese Annahme um zu verstehen, warum Operationen in einer Hash Map eine erwartete Laufzeit von $\mathcal{O}(1)$ haben:

Der Erwartungswert für die Anzahl der Elemente in einem Slot unter der Annahme von Uniform Hashing ist $n \cdot \frac{1}{m} = \frac{n}{m}$. Nehmen wir an, dass $n \in \mathcal{O}(m)$, also die Anzahl der Elemente $n$ maximal ein Vielfaches (Konstante als Faktor) von der Anzahl der Slots $m$ ist, so befinden sich vorraussichtlich $\leqslant \frac{c \cdot m}{m} = c$ Elemente in einer Liste. Da $c$ eine Konstante ist, befinden sich lediglich $\mathcal{O}(1)$ Elemente in einer Liste.
<!-- div style="text-align: right; font-size: 24px;">&#9633;</div -->

## Table Doubling

Das Problem ist, dass $n$ nicht vorhersagbar ist; schließlich können ständig Werte in das Dictionary eingefügt und entfernt werden. Ist $n$ nämlich viel größer als $m$, so sind die Listen, durch die iteriert werden muss, sehr lang und somit wäre die Datenstruktur ineffizient. Wie kann man also sicherstellen, dass $n \in \mathcal{O}(m)$ immer gilt? Hierfür wird eine Technik angewandt, die bereits von dynamischen Arrays bekannt ist, nämlich das Verdoppeln der Größe, sobald die Anzahl der Elemente zu groß wird. Man spricht hier von Table Doubling.

Table Doubling wird durchgeführt, sobald $n > m$. Dabei wird die Anzahl der Slots $m$ verdoppelt. Die Anzahl der Slots nach Table Doubling ist dann $m'=2m$. Um die Elemente nun auf $m'$ Slots zu verteilen, ist eine neue Hashfunktion nötig. Handelt es sich zunächst beispielsweise um die sehr einfache Hashfunktion $h: x \mapsto x \bmod m$, so wird sie entsprechend zu $h': x \mapsto x \bmod m'$ abgeändert. Die bereits eingefügten Elemente werden nun nach der neuen Hashfunktion $h'$ neu eingefügt, man spricht hier von Rehashing. Die Kosten für die ganze Table Doubling Operation betragen $\mathcal{O}(n)$. Analog wie für dynamische Arrays kann man aber zeigen, dass die Kosten amortisiert weiterhin lediglich $\mathcal{O}(1)$ betragen.

Durch Table Doubling ist sichergestellt, dass $n \in \mathcal{O}(m)$ immer gilt und somit die erwartete Laufzeit mit Uniform Hashing $\mathcal{O}(1)$ beträgt.

## Open Addressing

Open Addressing ist eine andere Variante, um eine Hash Map zu implementieren. Dabei gibt es keine Verkettung, sondern es wird immer maximal ein Element in einem Slot gespeichert. Somit muss $m \geqslant n$ gelten. 

__Insert__

Möchte man ein Element einfügen, das einen Hash-Wert hat, welcher einem Slot entspricht, der schon belegt ist, so berechnet man einen neuen Hash und fügt das Element an dieser Stelle ein, falls dieser Slot frei ist. Die Hashfunktion bei Open Addressing ist also eine spezielle Hashfunktion, die nicht nur den Schlüssel als Parameter nimmt, sondern auch die Zahl des Versuches, um den es sich handelt. Die Hashfunktion $h$ ist also von der Form 

$$
h: U \times \{0, 1, \dotsc, m-1\} \to \{0, 1, \dotsc, m-1\}.
$$

Eine entscheidende Anforderung an solch eine Hashfunktion ist, dass der Vektor 

$$
\begin{pmatrix}h(k, 0) & h(k, 1) & \cdots & h(k, m-1) \end{pmatrix}
$$ 

mit $k\in U$ eine Permutation des Vektors 

$$
\begin{pmatrix}0 & 1 & \cdots & m-1 \end{pmatrix}
$$ 

ist.

Dies bedeutet, dass man nach $m$ Versuchen jeden der $m$ Slots genau einmal "durchprobiert" hat. Unter der Bedingung $m \geqslant n$, findet man demnach in jedem Fall einen freien Slot.

__Search__

Um einen Schlüssel zu suchen, wendet man die Hashfunktion $h$ ebenfalls mit entsprechendem Versuchszähler an. Man beginnt mit der Versuchszahl 0. Hat man Glück und findet den Schlüssel an dem entsprechenden Slot, so kann man aufhören und hat das Element gefunden. Handelt es sich um einen __null__ Wert, so kann man auch aufhören, mit der Erkenntnis, dass sich der gesuchte Schlüssel nicht in der Hash Map befindet. Findet man an dem entsprechenden Slot, einen anderen Schlüssel, so muss man einen neuen Hash-Wert mit inkrementierten Versuchszähler berechnen. 

Dass man bei einem __null__ Wert aufhören kann, liegt daran, dass wenn der gesuchte Schlüssel sich in der Datenstruktur befände, man ihn genau an dieser Stelle eingefügt hätte. Da er sich aber nicht dort befindet, befindet er sich auch nirgendswo anders in der Hash Map.

__Remove__

Das Entfernen von Schlüsseln aus einer Hash Table mit Open Addressing stellt sich etwas problematisch dar. Entfernt man nämlich ein Element, so funktioniert die Suche nicht mehr, da die Probiersequenz, die zum eigentlichen Schlüssel führen soll, nun durch ein gelöschtes Element unterbrochen wurde und der Suchalgorithmus "denken" würde, der Schlüssel befindet sich nicht in der Hash Table. Aus diesem Grund darf man das Element nicht einfach entfernen, sondern man muss einen __deleted__-Marker einführen, der signalisiert, dass man weitersuchen muss, da sich hier mal ein Schlüssel befand. Der Suchalgorithmus bleibt prinzipiell der gleiche. Trifft man auf einen __null__-Wert, so kann man aufhören. Hat man den gesuchten Schlüssel gefunden, so kann man ebenfalls aufhören. Bei jedem anderen Wert - __deleted__-Marker eingeschlossen - muss man mit inkrementierten Versuchszähler weiterprobieren.

Nun stellt sich die Frage, wie man eine Hashfunktion entwickelt, die den Anforderungen von Open Addressing entspricht. Sie muss zusätzlich eine Versuchszahl als Parameter nehmen und alle $m$ Slots genau einmal zurückgeben.

### Linear Probing

Die naheliegendste Variante (lineares Sondieren) ist es, einfach beim nächsten Versuch um $i$ Slot(s) weiterzugehen, um auf einen freien Platz zum Schlüsseleintrag zu treffen. Also entwirft man folgende Hashfunktion, wobei $h'$ eine gewöhnliche einstellige Hashfunktion vom Typ $h':U \mapsto \{0, 1, \dotsc, m-1\}$ ist:

$$
h(k, i) = (h'(k) + i)\bmod m
$$

Diese Hashfunktion ist einfach, aber leider schlecht. Hat sich etwa ein *Cluster* in der Hash Map gebildet, d.h. ein Zustand, bei dem viele Slots unmittelbar hintereinander belegt sind, so muss man definitiv das komplette Cluster durchiterieren, bis man einen freien Platz findet. Zudem hat man jetzt dieses Cluster auch noch um 1 vergrößert. Je größer ein Cluster ist, desto größer ist auch die Wahrscheinlichkeit es beim ersten Versuch zu treffen. Genau genommen $\frac{k}{m}$, wenn $k$ die Größe des Clusters ist und $m$ die Anzahl der Slots in der Hash Map.

### Double Hashing

Double Hashing verwendet zwei zufällig ausgewählte Hashfunktionen $h_1$ und $h_2$. Die Hashfunktion $h$ wird folgendermaßen definiert:

$$
h(k, i) = \left( h_1(k) + i h_2(k) \right) \bmod m
$$

Damit sichergestellt ist, dass die Hashfunktion die erforderte Permutation erzeugt, müssen alle Werte von $h_2(k)$ und $m$ relativ prim, also teilerfremd, sein.

Diese Hashfunktion erzeugt eine wesentlich durchmischtere Verteilung der Hash-Werte, wodurch sich (wahrscheinlich) keine Cluster bilden und die somit besser geeignet ist.

## Hashfunktionen

Bisher wurde geklärt, wie man eine Hash Table implementiert. Nun ist zu klären, wie man eine passende Hashfunktion $h$, die vom Schlüsseluniversum auf $\{0, 1, \dotsc, m-1 \}$ abbildet, konstruiert. Im Folgenden werden drei Methoden vorgestellt.

### Divisionsmethode

$$
h(k) = k \bmod m
$$

Diese Hashfunktion ist sehr einfach, jedoch nicht sonderlich gut. Ist $m$ eine Zweierpotenz, was bei Table Doubling immer der Fall ist, so teilt es sich unter Umständen sehr viele Teiler mit dem Schlüssel, was dazu führt, dass bestimmte Hash-Werte unter der Menge $\{0, 1, \dotsc, m-1 \}$ besonders häufig vorkommen. Dies würde zu vielen Kollisionen führen, was die Datenstruktur ineffizient macht.

### Multiplikationsmethode

$$
h(k) = \left\lfloor \frac{A \cdot k \bmod 2^w}{2^{w-r}} \right\rfloor
$$

<img src="http://staff.ustc.edu.cn/~csli/graduate/algorithms/book6/229_a.gif" width="350">

Diese Hashfunktion multipliziert den Schlüssel mit einem Faktor $A$. $w$ ist die Wortbreite des Systems. $A$ ist aus der Menge $\{0, 1, \dotsc , 2^{w-1} \}$ zu wählen. Das Ergebnis der Multiplikation $A \cdot k$ besteht aus $2w$ Bits. Anschließend wird $\bmod 2^w$ gebildet, was dazu führt, dass nur die rechten Hälfte, also die rechten $w$ Bits weiter betrachtet werden. Danach wird durch $2^{w-r}$ geteilt, was einem Bitshift von $w-r$ Bits entspricht. Als Ergebnis bleibt ein Wert mit $r$ Bits übrig. $r$ ist dabei so zu wählen, dass gilt $m=2^r$.

Diese Hashfunktion ist besser, da hier ein Durchmischen stattfindet und somit die Hashwerte besser, also gleichmäßiger, verteilt sind.

### Universelles Hashing

Eine Möglichkeit für universelles Hashing ist die Klasse folgender Hashfunktionen:

$$
h(k) = ((a \cdot k + b) \bmod p) \bmod m
$$

$p$ ist dabei eine Primzahl, für die gilt $p \geqslant m$. Wird eine Hashfunktion benötigt, so werden $a$ und $b$ zufällig aus der Menge $\{0, 1, \dotsc, p-1\}$ gewählt. Damit die Hash-Werte uniform auf $\{0, 1, \dotsc, m-1\}$ verteilt werden, müssen $p$ und $m$ teilerfremd sein. Durch die Bedingung, dass $p$ eine Primzahl ist, ist dies gegeben.

Da $a$ und $b$ zufällig gewählt werden, handelt es sich hierbei um eine Menge bzw. Klasse $\mathcal{H}$ von Hashfunktionen, die aus $p^2$ Funktionen besteht. Die hier vorgestellte Klasse von Hashfunktionen ist $\approx 1$-universell.

Eine Klasse $\mathcal{H}$ von Hashfunktionen ist **$c$-universell**, wenn für alle $x, y \in U$ mit $x \neq y$ mit zufällig ausgewählten $h \in \mathcal{H}$ gilt, dass $Pr(h(x) = h(y)) \leqslant c \cdot \frac{1}{m}$.

Ist die Klasse $\mathcal{H}$ 1-universell, so ist bei zufällig ausgewähltem $h$ die Wahrscheinlichkeit einer Kollision $\leqslant \frac{1}{m}$. Anders fomulliert gibt es nicht mehr als $c \cdot \frac{\left| \mathcal{H} \right|}{m}$ Hashfunktionen, die für ein Schlüsselpaar den gleichen Hash-Wert liefern.

Auf den Beweis, dass die genannte Klasse von Hashfunktionen universell ist, wird verzichtet.

# String Matching

String Matching ist ein Problem, bei dem es darum geht, in einem String einen bestimmten Substring zu finden. Dies ist ein Problem, welches in der Praxis sehr häufig auftritt, beispielsweise in Texteditoren bei der Suche mit CTRL+F oder beim Linux Tool `grep`. In beiden Beispielen geht es darum, in einer sehr großen Datei, also einem String (Zeichenkette), einen bestimmten Substring zu finden, bzw. herauszufinden, ob dieser überhaupt vorkommt.

Ein naiver Ansatz ist es, jede Position der Zeichenkette der Länge $n$ als potenzielle Startposition des $k$-langen Substrings anzusehen und durch Zeichenvergleich ($k$ Zeichen) zu prüfen. $n$ ist die Länge des Strings und $k$ die Länge des Substrings. Dieser Algorithmus hat einen Zeitaufwand von $\mathcal{O}(nk)$.

In [3]:
def string_matching(str, substr):
    for i in range(len(str) - len(substr) + 1):
        match = True
        for j in range(len(substr)):
            if str[i + j] != substr[j]:
                match = False
        if match:
            return i
    return None


print(string_matching('Just using a naive string matching approach', 'string matching'))

19


## Karp-Rabin Algorithmus

Der Karp-Rabin Algorithmus ermöglicht es, das String Matching Problem in einer besseren Laufzeit als $\mathcal{O}(nk)$ zu lösen. Dabei wird das Prinzip von Hashes genutzt. Anstatt aber beim Durchiterieren jedesmal einen neuen Hash zu berechnen, nutzt man die Tatsache, dass sich beim Verschieben des Schiebefensters, welcher den Substring, der gerade untersucht wird, darstellt, sich nur das erste und letzte Zeichen ändern und die Mitte des Substrings gleich bleibt. Konstruiert man die Hashfunktion so, dass man den String als Zahl mit der Basis, die der Größe des Alphabets entspricht, auffasst, so kann man den Hash nach Entfernen des ersten Zeichens und Anfügen eines neuen Zeichens, mit Hilfe von einfachen Rechenoperationen in konstanter Zeit berechnen.

__Beispiel für einen Hash mit Basis 26: __
$h('hash') = 7 \cdot 26^3 + 0 \cdot 26^2 + 18 \cdot 26 + 7 \cdot 1 = 123507$

Fügt man jetzt z.B. das Zeichen x hinzu, so müsste man zunächst die Zahl $(7,0,18,7)_{26}$ mit der Basis multiplizieren, also erhält man $(7,0,18,7,0)_{26}$. Diese Operation ist äquivalent zur Multiplikation mit 10 im Dezimalsystem. Nun addiert man den Wert des neuen Zeichens, nämlich 23 und erhält $(7,0,18,7,23)_{26}$.

Nun benötigt man eine Rechenoperation, mit der man das erste Zeichen entfernen kann. Man muss also die erste Ziffer abschneiden. Dies führt man durch, indem man die Hash-Zahl modulo $\text{Basis}^{\text{Länge des Strings - 1}}$ nimmt. In diesem Fall $\bmod 26^4$ und man erhält $(0,18,7,23)_{26}$.

Mit Hilfe dieser Arithmetik kann man nun eine Datenstruktur names **Rolling Hash** implementieren, die in konstanter Zeit ein Zeichen anfügt, in konstanter Zeit das erste Zeichen entfernt und in konstanter Zeit den aktuellen Hash-Wert zurückgibt.

Der Algorithmus iteriert nun durch $n$ Zeichen des Strings und vergleicht den Rolling Hash-Wert *des Substrings ab dem entsprechenden Index* mit dem Rolling Hash-Wert *des gesuchten Substrings*, der im Voraus berechnet wurde. Da die Basis nicht der Größe des Alphabets entspricht, dies auch für UNICODE-Zeichen nicht praktikabel wäre, kommt es zu Kollisionen, d.h. unterschiedliche Strings können den gleichen Hash-Wert haben. Dadurch muss man im positiven Fall, d.h. die Hash-Werte stimmen überein, beide Strings noch Zeichen für Zeichen überprüfen. Da aber Kollisionen sehr selten sind, hat das keinen bedeutenden Einfluss auf die erwartete Laufzeit des Algorithmus.

Die erwartete Laufzeit des Algorithmus beträgt $\mathcal{O}(n+k)$, da man zunächst einen Aufwand von $k$ hat, um den Rolling Hash des gesuchten Substrings zu berechnen und anschließend $n$-mal (erwarteten) konstanten Aufwand hat, um den Rolling Hash mit den __append__ und __skip__ Operationen anzupassen und ihn entsprechend abzugleichen. Dieser Algorithmus ist im Vergleich zum naiven Brute-Force Ansatz besonders effizient, wenn $k$, also die Länge des Substrings, groß ist.

In [4]:
class RollingHash:
    def __init__(self):
        self.string = ''
        self.base = 256
        self.hash = 0

    def __hash__(self):
        return self.hash

    def append(self, c):
        self.hash *= self.base
        self.hash += ord(c) % self.base
        self.string += c

    def skip(self):
        if len(self.string) > 0:
            self.string = self.string[1:]
            self.hash %= self.base ** len(self.string)
        else:
            raise Exception('string is empty')


def karp_rabin(str, substr):
    if len(substr) <= len(str):
        rolling_hash_substr = RollingHash()
        rolling_hash_str = RollingHash()
        for i in range(len(substr)):
            rolling_hash_substr.append(substr[i])
            rolling_hash_str.append(str[i])
        hash_value_substr = hash(rolling_hash_substr)
        if hash_value_substr == hash(rolling_hash_str):
            if rolling_hash_str.string == substr:
                return 0
        for i in range(len(substr), len(str)):
            rolling_hash_str.skip()
            rolling_hash_str.append(str[i])
            if hash_value_substr == hash(rolling_hash_str):
                if rolling_hash_str.string == substr:
                    return i - len(substr) + 1
    return None


print(karp_rabin('Karp Rabin is awesome.', 'Rabin'))

5
